# Homework 2

## FINM 36700 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

## HBS Case

### *ProShares Hedge Replication ETF*

### HW Group A 33

* Rain Condie
* Xuzhou Ding
* Oakley Liu
* Jose Luna Rozas
* Akhilesh Narayan
* Orianna Wang

***

# 1. The ProShares ETF Product

**Section 1 is not graded**, and you do not need to submit your answers. But you are encouraged to think about them, and we will discuss them.

## 1. Alternative ETFs

Describe the two types of investments referenced by this term.

## 2. Hedge Funds.

a. Using just the information in the case, what are two measures by which hedge funds are an attractive investment?

b. What are the main benefits of investing in hedge funds via an ETF instead of directly?

## 3. The Benchmarks

a. Explain as simply as possible how HFRI, MLFM, MLFM-ES, and HDG differ in their construction and purpose.

b. How well does the Merrill Lynch Factor Model (MLFM) track the HFRI?

c. In which factor does the MLFM have the largest loading? (See a slide in Exhibit 1.)

d. What are the main concerns you have for how the MLFM attempts to replicate the HFRI?

## 4. The HDG Product

a. What does ProShares ETF, HDG, attempt to track? Is the tracking error small?

b. HDG is, by construction, delivering beta for investors. Isn't the point of hedge funds to generate alpha? Then why would HDG be valuable?

c. The fees of a typical hedge-fund are 2% on total assets plus 20% of excess returns if positive. HDG's expense ratio is roughly 1% on total assets. What would their respective net Sharpe Ratios be, assuming both have a gross excess returns of 10% and volatility of 20%?

***

# 2.  Analyzing the Data

Use the data found on Canvas, in <b>'proshares analysis data.xlsx'</b>. 

It has monthly data on financial indexes and ETFs from `Aug 2011` through `Sep 2024`.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import skew,kurtosis

filename = r'C:\Users\josel\Documents\GitHub\finm-portfolio-2024\data\proshares_analysis_data.xlsx'
# filename = r"C:\Users\josel\Downloads\multi_asset_etf_data (1).xlsx"
data_hedge_fund_series = pd.read_excel(filename,sheet_name="hedge_fund_series",index_col=[0])
data_merrill_factors = pd.read_excel(filename,sheet_name="merrill_factors",index_col=[0])

## 1. 

For the series in the "hedge fund series" tab, report the following summary statistics:
* mean
* volatility
* Sharpe ratio

Annualize these statistics.

In [10]:
def get_metrics(port_metrics,weights = [],adj_factor = 12):
    if len(weights) == 0:
        port_metrics_r = pd.DataFrame({"Mean": port_metrics.mean()*adj_factor,"Volatility":port_metrics.std()*np.sqrt(adj_factor)})
        port_metrics_r["Sharpe_Ratio"] = (port_metrics.mean() / port_metrics.std()) * np.sqrt(adj_factor)
        port_metrics_r["Skew"] = skew(port_metrics)
        port_metrics_r["Excess Kurtosis"] = kurtosis(port_metrics, fisher=True, bias=False)    
    else:
        port_metrics = port_metrics @ weights
        port_metrics_r = pd.DataFrame({"Mean": port_metrics.mean()*adj_factor,"Volatility":port_metrics.std()*np.sqrt(adj_factor)})
        port_metrics_r["Sharpe_Ratio"] = (port_metrics.mean() / port_metrics.std()) * np.sqrt(adj_factor)
        port_metrics_r["Skew"] = skew(port_metrics)
        port_metrics_r["Excess Kurtosis"] = kurtosis(port_metrics, fisher=True, bias=False)
    return port_metrics_r

def VaR_CVaR_Drawdown_metrics(data_daily_return):
    result = pd.DataFrame()
    for asset in data_daily_return.columns:

        data_aux = data_daily_return[[asset]].copy()

            
        VaR = np.percentile(sorted(data_aux.values),q = 5)
        CVaR = data_aux[data_aux[asset] <= VaR].mean().values[0]

        data_aux_acum_return = (data_aux + 1).cumprod()
        data_aux_max_cum_return = data_aux_acum_return.cummax()
        data_aux_drawdown = ((data_aux_acum_return-data_aux_max_cum_return)/data_aux_max_cum_return)
        max_drawdown = data_aux_drawdown.min().values[0]
        max_drawdown_date = data_aux_drawdown.idxmin().values[0]
        peak_idx = data_aux_max_cum_return.idxmax().values[0]

        recovery_idx = data_aux_drawdown[data_aux_drawdown.idxmin().values[0]:].gt(-0.00001).idxmax().values[0]

        aux_result = pd.DataFrame([[VaR,CVaR,max_drawdown,max_drawdown_date,peak_idx,recovery_idx,(recovery_idx - max_drawdown_date)/ np.timedelta64(1, 'D')]], columns= ["VaR","CVaR","Max Drawdown","Bottom","Peak","Recovery","Duration (days)"], index = [asset])
        result = pd.concat([result,aux_result],axis=0)

    return result,data_aux_drawdown

def get_metrics_all(returns):
    metrics1 = get_metrics(returns)
    metrics2,_ = VaR_CVaR_Drawdown_metrics(returns)
    return pd.merge(metrics1,metrics2, left_index= True, right_index=True, how = "left")

In [4]:
data_hedge_fund_series.kurt()

HFRIFWI Index     5.763074
MLEIFCTR Index    1.631580
MLEIFCTX Index    1.595863
HDG US Equity     1.759250
QAI US Equity     1.605934
dtype: float64

In [11]:
get_metrics(data_hedge_fund_series)

,Mean,Volatility,Sharpe_Ratio,Skew,Excess Kurtosis
HFRIFWI Index,0.048295,0.059476,0.812004,-0.954148,5.763074
MLEIFCTR Index,0.036286,0.056136,0.646398,-0.262868,1.631580
MLEIFCTX Index,0.034605,0.055980,0.618172,-0.248887,1.595863
HDG US Equity,0.024812,0.058408,0.424805,-0.251781,1.759250
QAI US Equity,0.025653,0.050009,0.512971,-0.430637,1.605934


## 2.

For the series in the "hedge fund series" tab, calculate the following statistics related to tail-risk.
* Skewness
* Excess Kurtosis (in excess of 3)
* VaR (.05) - the fifth quantile of historic returns
* CVaR (.05) - the mean of the returns at or below the fifth quantile
* Maximum drawdown - include the dates of the max/min/recovery within the max drawdown period.

There is no need to annualize any of these statistics.

In [4]:
get_metrics_all(data_hedge_fund_series)

,Mean,Volatility,Sharpe_Ratio,Skew,Excess Kurtosis,VaR,CVaR,Max Drawdown,Bottom,Peak,Recovery,Duration (days)
HFRIFWI Index,0.048295,0.059476,0.812004,-0.954148,5.544571,-0.024293,-0.037478,-0.115473,2020-03-31,2024-09-30,2020-08-31,153.0
MLEIFCTR Index,0.036286,0.056136,0.646398,-0.262868,1.542667,-0.028189,-0.035857,-0.124302,2022-09-30,2024-09-30,2024-02-29,517.0
MLEIFCTX Index,0.034605,0.055980,0.618172,-0.248887,1.508071,-0.028095,-0.035816,-0.124388,2022-09-30,2024-09-30,2024-02-29,517.0
HDG US Equity,0.024812,0.058408,0.424805,-0.251781,1.666333,-0.030932,-0.037575,-0.140720,2022-09-30,2024-09-30,2024-07-31,670.0
QAI US Equity,0.025653,0.050009,0.512971,-0.430637,1.517826,-0.017895,-0.032695,-0.137716,2022-09-30,2024-09-30,2024-02-29,517.0


## 3. 

For the series in the "hedge fund series" tab, run a regression of each against SPY (found in the "merrill factors" tab.) Include an intercept. Report the following regression-based statistics:

$$ r_i = \alpha + \beta*r_m + \epsilon $$
* Market Beta : $\beta$
* Treynor Ratio: $(E[r]-r_f)/ \beta$
* Information ratio: $\alpha / \sigma_\epsilon$ 

Annualize these three statistics as appropriate.

In [5]:
import statsmodels.api as sm

def benchmark_regresion(data,benchmark = "SPY US Equity",adj = 12):
    result = pd.DataFrame()
    X = sm.add_constant(data[benchmark])
    for asset in data.drop([benchmark],axis=1).columns:
        X = sm.add_constant(data[benchmark])
        y = data[asset]
        mod = sm.OLS(y, X).fit()
        inter, beta = mod.params.values[0], mod.params.values[1]
        rsquare = mod.rsquared
        std_errors= mod.resid.std()
        TR = (y.mean()/beta)*adj
        IR = (inter/std_errors)*np.sqrt(adj)
        aux_result = pd.DataFrame([[inter*adj,beta,rsquare,std_errors,y.mean()*adj,TR,IR]],columns=["Alpha","Beta","R-square","std_errors","R_mean","Treynor Ratio","Information Ratio"], index = [asset])
        result = pd.concat([result,aux_result],axis=0)
    return result
     

In [6]:
data_hedge_fund_series_wBench = data_hedge_fund_series.merge(data_merrill_factors[["SPY US Equity"]],left_index=True,right_index=True, how = "left")

In [7]:
benchmark_regresion(data_hedge_fund_series_wBench)

,Alpha,Beta,R-square,std_errors,R_mean,Treynor Ratio,Information Ratio
HFRIFWI Index,-0.001145,0.346131,0.711480,0.009222,0.048295,0.139528,-0.035836
MLEIFCTR Index,-0.012761,0.343383,0.786051,0.007496,0.036286,0.105672,-0.491479
MLEIFCTX Index,-0.014269,0.342171,0.784862,0.007495,0.034605,0.101134,-0.549547
HDG US Equity,-0.025474,0.352053,0.763208,0.008205,0.024812,0.070478,-0.896268
QAI US Equity,-0.016965,0.298376,0.747822,0.007250,0.025653,0.085976,-0.675558


## 4. 

Discuss the previous statistics, and what they tell us about...

* the differences between SPY and the hedge-fund series?
* which performs better between HDG and QAI.
* whether HDG and the ML series capture the most notable properties of HFRI.

## 5. 

Report the correlation matrix for these assets.
* Show the correlations as a heat map.
* Which series have the highest and lowest correlations?

## 6.

Replicate HFRI with the six factors listed on the "merrill factors" tab. Include a constant, and run the unrestricted regression,

$\newcommand{\hfri}{\text{hfri}}$
$\newcommand{\merr}{\text{merr}}$

$$\begin{align}
r^{\hfri}_{t} &= \alpha^{\merr} + x_{t}^{\merr}\beta^{\merr} + \epsilon_{t}^{\merr}\\[5pt]
\hat{r}^{\hfri}_{t} &= \hat{\alpha}^{\merr} + x_{t}^{\merr}\hat{\beta}^{\merr}
\end{align}$$

Note that the second equation is just our notation for the fitted replication.

a. Report the intercept and betas.
b. Are the betas realistic position sizes, or do they require huge long-short positions?
c. Report the R-squared.
d. Report the volatility of $\epsilon^{\merr}$, the tracking error.

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import skew,kurtosis,norm
import statsmodels.api as sm


In [29]:
def Linear_Factor_Descomposition(data, y_asset,x_asset, adj = 12, constant = True):

    Y = data[y_asset]
    X = data[x_asset]

    if constant:
        X = sm.add_constant(X)

    mod = sm.OLS(Y, X).fit()
    inter = mod.params.values[0]

    rsquare = mod.rsquared
    std_errors= mod.resid.std()
    tracking_error = mod.resid.std() * np.sqrt(adj)

    metrics = pd.DataFrame([[inter,inter*adj,rsquare,std_errors,tracking_error]],columns=["Alpha","Alpha Adj","R-square","std_errors","tracking_error"], index = [y_asset])
    return pd.DataFrame(mod.params,columns=["Parameters"]), metrics,mod.summary()

In [21]:
data_regression = pd.merge(data_hedge_fund_series[["HFRIFWI Index"]],data_merrill_factors, left_index=True,right_index=True, how = "left")

In [17]:
data_merrill_factors.columns

Index(['SPY US Equity', 'USGG3M Index', 'EEM US Equity', 'EFA US Equity',
       'EUO US Equity', 'IWM US Equity'],
      dtype='object')

In [32]:
regresors, metrics, summary = Linear_Factor_Descomposition(data_regression, "HFRIFWI Index",x_asset = ['SPY US Equity', 'USGG3M Index', 'EEM US Equity', 'EFA US Equity',
       'EUO US Equity', 'IWM US Equity'], adj = 12, constant = True)

In [31]:
regresors

,Parameters
const,0.001235
SPY US Equity,0.036483
USGG3M Index,0.146336
EEM US Equity,0.081611
EFA US Equity,0.084638
EUO US Equity,0.028806
IWM US Equity,0.145111


In [33]:
metrics

,Alpha,Alpha Adj,R-square,std_errors,tracking_error
HFRIFWI Index,0.001235,0.014824,0.839676,0.006875,0.023815


## 7.

Let's examine the replication out-of-sample (OOS).

Starting with $t = 61$ month of the sample, do the following:

* Use the previous 60 months of data to estimate the regression equation. 
This gives time-t estimates of the regression parameters, $\tilde{\alpha}^{\merr}_{t}$ and $\tilde{\beta}^{\merr}_{t}$.

* Use the estimated regression parameters, along with the time-t regressor values, $x^{\merr}_{t}$, calculate the time-t replication value that is, with respect to the regression estimate, built "out-of-sample" (OOS).

$$\hat{r}^{\hfri}_{t} \equiv \tilde{\alpha}^{\merr} + (x_{t}^{\merr})'\tilde{\beta}^{\merr}$$

* Step forward to $t = 62$, and now use $t = 2$ through $t = 61$ for the estimation. Re-run the steps above, and continue this process throughout the data series. Thus, we are running a rolling, 60-month regression for each point-in-time.

How well does the out-of-sample replication perform with respect to the target?

## 8.

We estimated the replications using an intercept. Try the full-sample estimation, but this time without an intercept.

$$\begin{align}
r^{\hfri}_{t} &= \alpha^{merr} + x_{t}^{\merr}\beta^{\merr} + \epsilon_{t}^{\merr}\\[5pt]
\check{r}^{\hfri}_{t} &= \check{\alpha}^{\merr} + x_{t}^{\merr}\check{\beta}^{\merr}
\end{align}$$

Report

* the regression beta. How does it compare to the estimated beta with an intercept, $\hat{\beta}^{\merr}$?

* the mean of the fitted value, $\check{r}^{\hfri}_{t}$. How does it compare to the mean of the HFRI?

* the correlations of the fitted values, $\check{r}^{\hfri}_{t}$ to the HFRI. How does the correlation compare to that of the fitted values with an intercept, $\hat{r}^{\hfri}_{t}$

Do you think Merrill and ProShares fit their replicators with an intercept or not?

***

# 3.  Extensions
<i>This section is not graded, and you do not need to submit it. Still, we may discuss it in class, in which case, you would be expected to know it.

## 1. 

Merrill constrains the weights of each asset in its replication regression of HFRI. Try constraining your weights by re-doing 2.6.

* Use Non-Negative Least Squares (NNLS) instead of OLS.
* Go further by using a Generalized Linear Model to put separate interval constraints on each beta, rather than simply constraining them to be non-negative.

#### Hints
* Try using LinearRegression in scikit-learn with the parameter `positive=True`. 
* Try using GLM in statsmodels.

## 2. 

Let's decompose a few other targets to see if they behave as their name suggests.

* Regress HEFA on the same style factors used to decompose HFRI. Does HEFA appear to be a currency-hedged version of EFA?

* Decompose TRVCI with the same style factors used to decompose HFRI. The TRVCI Index tracks venture capital funds--in terms of our styles, what best describes venture capital?

* TAIL is an ETF that tracks SPY, but that also buys put options to protect against market downturns. Calculate the statistics in questions 2.1-2.3 for TAIL. Does it seem to behave as indicated by this description? That is, does it have high correlation to SPY while delivering lower tail risk?

## 3. 

The ProShares case introduces Levered ETFs. ProShares made much of its name originally through levered, or "geared" ETFs.

Explain conceptually why Levered ETFs may track their index well for a given day but diverge over time. How is this exacerbated in volatile periods like 2008?

## 4.

Analyze SPXU and UPRO relative to SPY.
- SPXU is ProShares -3x SPX ETF.
- UPRO is ProShres +3x SPX ETF.

Questions:
* Analyze them with the statistics from 2.1-2.3. 

* Do these two ETFs seem to live up to their names?

* Plot the cumulative returns of both these ETFs along with SPY.

* What do you conclude about levered ETFs?

***